# Example for qutrit GST
This notebook demonstrates how to construct the operation sequences and perform the analysis for qutrit GST when the model consists of symmetric $\pi/2$-rotations on each single qubit separately, `X`, `Y` and a 2-qubit Molmer-Sorenson gate which rotates around the `XX` axis by $\pi/2$.

In [1]:
import pygsti
from pygsti.models import qutrit

from numpy import pi, array
import pickle

import numpy as np

First, we construct the target model.  This functionality is built into pyGSTi, so we just need to specify the single-qubit and M-S angles.

In [2]:
target_model = qutrit.create_qutrit_model(error_scale=0, x_angle=pi/2, y_angle=pi/2, ms_global=pi/2, ms_local=0, basis="qt")
#print(target_model)

Now construct the operation sequences needed by GST.  These fiducials and germs have been computed ahead of time and the results are used to construct the operation sequence lists below.  Then we construct an empty dataset containing all of the necessary experimental sequences which can serve as a template for the actual experimental results.

In [3]:
fiducialPrep = pygsti.circuits.to_circuits(
    [(),('Gy',),('Gx',),('Gm',),
    ('Gx','Gx'), ('Gm','Gy'),('Gm','Gx'),
    ('Gy','Gy','Gy'),('Gx','Gx','Gx')])

fiducialMeasure = pygsti.circuits.to_circuits(
    [(),('Gy',),('Gx',),('Gm',),
    ('Gy','Gm'),('Gx','Gm')])

maxLengths = [1,2,4]

germs = pygsti.circuits.to_circuits(
[('Gi',),
 ('Gy',),
 ('Gx',),
 ('Gm',),
 ('Gi', 'Gy'),
 ('Gi', 'Gx'),
 ('Gi', 'Gm'),
 ('Gy', 'Gx'),
 ('Gy', 'Gm'),
 ('Gx', 'Gm'),
 ('Gi', 'Gi', 'Gy'),
 ('Gi', 'Gi', 'Gx'),
 ('Gi', 'Gi', 'Gm'),
 ('Gi', 'Gy', 'Gy'),
 ('Gi', 'Gy', 'Gx'),
 ('Gi', 'Gy', 'Gm'),
 ('Gi', 'Gx', 'Gy'),
 ('Gi', 'Gx', 'Gx'),
 ('Gi', 'Gx', 'Gm'),
 ('Gi', 'Gm', 'Gy'),
 ('Gi', 'Gm', 'Gx'),
 ('Gi', 'Gm', 'Gm'),
 ('Gy', 'Gy', 'Gx'),
 ('Gy', 'Gy', 'Gm'),
 ('Gy', 'Gx', 'Gx'),
 ('Gy', 'Gx', 'Gm'),
 ('Gy', 'Gm', 'Gx'),
 ('Gy', 'Gm', 'Gm'),
 ('Gx', 'Gx', 'Gm'),
 ('Gx', 'Gm', 'Gm')])

In [4]:
#Note above construction is now a "standard" qutrit model
from pygsti.modelpacks.legacy import stdQT_XYIMS
target_model = stdQT_XYIMS.target_model()
fiducialPrep = stdQT_XYIMS.prepStrs
fiducialMeasure = stdQT_XYIMS.effectStrs
germs = stdQT_XYIMS.germs_lite
maxLengths = [1,2,4]

In [5]:
print("%d prep fiducials" % len(fiducialPrep))
print("%d meas fiducials" % len(fiducialMeasure))
print("%d germs" % len(germs))

9 prep fiducials
7 meas fiducials
31 germs


In [6]:
#generate data template
expList = pygsti.circuits.create_lsgst_circuits(target_model.operations.keys(), fiducialPrep, fiducialMeasure, germs,  maxLengths)
pygsti.io.write_empty_dataset("example_files/dataTemplate_qutrit_maxL=4.txt", expList, "## Columns = 0bright count, 1bright count, 2bright count")

At this point **STOP** and create/fill a dataset file using the template written in the above cell.  Then proceed with the lines below to run GST on the data and create (hopefully useful) reports telling you about your gates.

In [7]:
mdl_datagen = target_model.depolarize(op_noise=0.05)
DS = pygsti.data.simulate_data(mdl_datagen, expList, 500, sample_error='multinomial', seed=2018)

In [8]:
#DS = pygsti.io.load_dataset('PATH_TO_YOUR_DATASET',cache=True) # (cache=True speeds up future loads)

In [9]:
#Run qutrit GST... which could take a while on a single CPU.  Please adjust memLimit to machine specs 
# (now 3GB; usually set to slightly less than the total machine memory)
target_model.sim = "matrix"
result = pygsti.run_stdpractice_gst(DS,target_model,fiducialPrep,fiducialMeasure,germs,maxLengths,
                                     verbosity=4, comm=None, mem_limit=3*(1024)**3, modes="TP,CPTP")

--- Circuit Creation ---
   2017 circuits created
   Dataset has 2017 entries: 2017 utilized, 0 requested circuits were missing
-- Std Practice:  Iter 1 of 2  (TP) --: 
  --- Iterative GST: Iter 1 of 3  210 circuits ---: 
    Layout creation w/mem limit = 3.00GB
    MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter directions.
       1 atoms, parameter block size limits (None,)
    *** Distributing 1 atoms to 1 atom-processing groups (1 cores) ***
        More atom-processors than hosts: each host gets ~1 atom-processors
        Atom-processors already occupy a single node, dividing atom-processor into 1 param-processors.
    *** Divided 1-host atom-processor (~1 procs) into 1 param-processing groups ***
    --- chi2 GST ---


/Users/enielse/pyGSTi/pygsti/protocols/gst.py:292: UserWarning: Failed to create a processor spec for GST experiment design!
  _warnings.warn("Failed to create a processor spec for GST experiment design!")


      --- Outer Iter 0: norm_f = 2.44942e+06, mu=1, |x|=5.88784, |J|=48997.9
      --- Outer Iter 1: norm_f = 454.154, mu=89186.6, |x|=5.71913, |J|=2540.12
      --- Outer Iter 2: norm_f = 391.887, mu=29728.9, |x|=5.71325, |J|=2433.17
      --- Outer Iter 3: norm_f = 354.498, mu=9909.62, |x|=5.70711, |J|=2410.6
      --- Outer Iter 4: norm_f = 313.31, mu=3303.21, |x|=5.6964, |J|=2414.53
      --- Outer Iter 5: norm_f = 265.397, mu=1101.07, |x|=5.67739, |J|=2457.22
      --- Outer Iter 6: norm_f = 229.442, mu=612.224, |x|=5.65579, |J|=5220.83
      --- Outer Iter 7: norm_f = 205.854, mu=204.075, |x|=5.64488, |J|=5217.41
      --- Outer Iter 8: norm_f = 197.162, mu=68.0249, |x|=5.63821, |J|=5212.91
      --- Outer Iter 9: norm_f = 194.835, mu=22.675, |x|=5.63554, |J|=5207.24
      --- Outer Iter 10: norm_f = 194.574, mu=7.55832, |x|=5.63469, |J|=5204.15
      --- Outer Iter 11: norm_f = 194.564, mu=2.51944, |x|=5.63455, |J|=5203.45
      --- Outer Iter 12: norm_f = 194.564, mu=0.839814, 

/Users/enielse/pyGSTi/pygsti/protocols/gst.py:1259: UserWarning: Could not create target model for gauge opt. from edesign - falling back to initial model!
  _warnings.warn(("Could not create target model for gauge opt. from edesign"


-- Std Practice:  Iter 2 of 2  (CPTP) --: 
  --- Iterative GST: Iter 1 of 3  210 circuits ---: 
    Layout creation w/mem limit = 3.00GB
    MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter directions.
       1 atoms, parameter block size limits (None,)
    *** Distributing 1 atoms to 1 atom-processing groups (1 cores) ***
        More atom-processors than hosts: each host gets ~1 atom-processors
        Atom-processors already occupy a single node, dividing atom-processor into 1 param-processors.
    *** Divided 1-host atom-processor (~1 procs) into 1 param-processing groups ***
    --- chi2 GST ---
      --- Outer Iter 0: norm_f = 2.44942e+06, mu=1, |x|=6.9282e-08, |J|=1205.38
      --- Outer Iter 1: norm_f = 1568.14, mu=37.6409, |x|=0.302157, |J|=1422.57
      --- Outer Iter 2: norm_f = 520.96, mu=842.25, |x|=0.381817, |J|=1240.23
      --- Outer Iter 3: norm_f = 421.827, mu=821.677, |x|=0.412, |J|=1251
      --- Outer Iter 4: norm_f = 351.77, mu=

      --- Outer Iter 94: norm_f = 264.625, mu=1320.13, |x|=0.476277, |J|=1332.65
      --- Outer Iter 95: norm_f = 264.624, mu=1320.13, |x|=0.476277, |J|=1332.67
      --- Outer Iter 96: norm_f = 264.623, mu=1317.97, |x|=0.476279, |J|=1332.69
      --- Outer Iter 97: norm_f = 264.621, mu=1299.92, |x|=0.476279, |J|=1332.71
      --- Outer Iter 98: norm_f = 264.62, mu=1247.12, |x|=0.476281, |J|=1332.73
      --- Outer Iter 99: norm_f = 264.619, mu=1165.77, |x|=0.476281, |J|=1332.75
      --- Outer Iter 100: norm_f = 264.617, mu=1105.03, |x|=0.476283, |J|=1332.77
      --- Outer Iter 101: norm_f = 264.616, mu=1092.38, |x|=0.476285, |J|=1332.79
      --- Outer Iter 102: norm_f = 264.615, mu=1092.38, |x|=0.476287, |J|=1332.82
      --- Outer Iter 103: norm_f = 264.614, mu=1116.83, |x|=0.476289, |J|=1332.83
      --- Outer Iter 104: norm_f = 264.613, mu=1253.25, |x|=0.476292, |J|=1332.86
      --- Outer Iter 105: norm_f = 264.612, mu=1397.41, |x|=0.476293, |J|=1332.87
      --- Outer Iter 10

      --- Outer Iter 46: norm_f = 984.979, mu=683.705, |x|=0.478513, |J|=2364.31
      --- Outer Iter 47: norm_f = 984.973, mu=690.651, |x|=0.478468, |J|=2364.42
      --- Outer Iter 48: norm_f = 984.967, mu=696.381, |x|=0.478427, |J|=2364.5
      --- Outer Iter 49: norm_f = 984.961, mu=701.149, |x|=0.478386, |J|=2364.6
      --- Outer Iter 50: norm_f = 984.957, mu=705.066, |x|=0.478349, |J|=2364.68
      --- Outer Iter 51: norm_f = 984.952, mu=708.344, |x|=0.478311, |J|=2364.77
      --- Outer Iter 52: norm_f = 984.948, mu=711.075, |x|=0.478277, |J|=2364.85
      --- Outer Iter 53: norm_f = 984.944, mu=713.398, |x|=0.478242, |J|=2364.93
      --- Outer Iter 54: norm_f = 984.941, mu=715.369, |x|=0.47821, |J|=2365.01
      --- Outer Iter 55: norm_f = 984.938, mu=717.07, |x|=0.478177, |J|=2365.09
      --- Outer Iter 56: norm_f = 984.935, mu=718.528, |x|=0.478147, |J|=2365.16
      --- Outer Iter 57: norm_f = 984.932, mu=719.789, |x|=0.478116, |J|=2365.24
      --- Outer Iter 58: norm_f 

      --- Outer Iter 65: norm_f = 3772.52, mu=257.77, |x|=0.462912, |J|=4512.81
      --- Outer Iter 66: norm_f = 3772.52, mu=249.934, |x|=0.462949, |J|=4512.7
      Least squares message = Both actual and predicted relative reductions in the sum of squares are at most 1e-06
    Sum of Chi^2 = 3772.52 (4034 data params - 432 (approx) model params = expected mean of 3602; p-value = 0.0235163)
    Completed in 335.1s
    Iteration 3 took 335.6s
    
    Last iteration:
    --- dlogl GST ---
      --- Outer Iter 0: norm_f = 1894.42, mu=1, |x|=0.462949, |J|=3188.99
      --- Outer Iter 1: norm_f = 1892.03, mu=118.718, |x|=0.462425, |J|=3194.04
      --- Outer Iter 2: norm_f = 1891.76, mu=39.5726, |x|=0.462704, |J|=3195.2
      --- Outer Iter 3: norm_f = 1891.68, mu=28.1628, |x|=0.463194, |J|=3195.42
      --- Outer Iter 4: norm_f = 1891.64, mu=28.1606, |x|=0.463572, |J|=3195.2
      --- Outer Iter 5: norm_f = 1891.64, mu=34.4644, |x|=0.463827, |J|=3194.9
      --- Outer Iter 6: norm_f = 18

In [10]:
#Create a report
ws = pygsti.report.construct_standard_report(
    result, "Example Qutrit Report", verbosity=3
).write_html('example_files/sampleQutritReport', auto_open=True, verbosity=3)

Running idle tomography
Computing switchable properties


/Users/enielse/pyGSTi/pygsti/report/factory.py:1245: UserWarning: Idle tomography failed:
operands could not be broadcast together with shapes (9,) (16,) 
  _warnings.warn("Idle tomography failed:\n" + str(e))
/Users/enielse/pyGSTi/pygsti/report/workspacetables.py:752: UserWarning:

Failed gauge-robust decomposition of Gi op:


